In [10]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import copy
import random

# Dim reduction tools
import umap
from sklearn.linear_model import LinearRegression
import torch
import torch.utils.data
from torch import optim
from torch.autograd import Variable
import torch.nn as nn
from torch.utils.data import TensorDataset
from sklearn.model_selection import ShuffleSplit

# 
from utils import *
import segypy 

#### Copy paste "utils.py" because it wont load/import:

In [17]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = False  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = False

    return True

def load_seismic(filename, inlines=[1300, 1502, 2], xlines=[1500, 2002, 2]):
    inl = np.arange(*inlines)
    crl = np.arange(*xlines)
    seis, header, trace_headers = segypy.readSegy(filename)
    amplitude = seis.reshape(header['ns'], inl.size, crl.size)
    lagtime = trace_headers['LagTimeA'][0]*-1
    twt = np.arange(lagtime, header['dt']/1e3*header['ns']+lagtime, header['dt']/1e3)
    return amplitude, twt


def load_horizon(filename, inlines=[1300, 1502, 2], xlines=[1500, 2002, 2]):
    inl = np.arange(*inlines)
    crl = np.arange(*xlines)
    hrz = np.recfromtxt(filename, names=['il','xl','z'])
    horizon = np.zeros((len(inl), len(crl)))
    for i, idx in enumerate(inl):
        for j, xdx in enumerate(crl):
            time = hrz['z'][np.where((hrz['il']== idx) & (hrz['xl'] == xdx))]
            if len(time) == 1:
                horizon[i, j] = time 

    return horizon

def colorbar(mappable):
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    return fig.colorbar(mappable, cax=cax)

def interpolate_horizon(horizon):
    points = []
    wanted = []
    for i in range(horizon.shape[0]):
        for j in range(horizon.shape[1]):
            if horizon[i, j] != 0.:
                points.append([i, j, horizon[i, j]])
            else:
                wanted.append([i, j])
    
    points = np.array(points)
    zs2 = scipy.interpolate.griddata(points[:, 0:2], points[:, 2], wanted, method="cubic")
    for p, val in zip(wanted, zs2):
        horizon[p[0], p[1]] = val
    
    return horizon

def plot_section_horizon_and_well(ax, amplitude, horizon, twt, inline=38, well_pos=276//2):
    hrz_idx = [np.abs(twt-val).argmin() for val in horizon[inline, :]]
    
    h_bin = np.zeros((amplitude.shape[0], amplitude.shape[2]))
    for i, val in enumerate(hrz_idx):
        h_bin[val, i] = 1

    clip = abs(np.percentile(amplitude, 0.8))
    ax.imshow(amplitude[:, inline], cmap="Greys", vmin=-clip, vmax=clip)
    ax.plot(range(len(hrz_idx)), hrz_idx, linewidth=5, color="black")
    ax.axvline(well_pos, color="red", linewidth=5)

def flatten_on_horizon(amplitude, horizon, twt, top_add=12, below_add=52):
    traces = np.zeros((horizon.shape[0], horizon.shape[1], top_add+below_add))
    for i in range(horizon.shape[0]):
        hrz_idx = [np.abs(twt-val).argmin() for val in horizon[i, :]]
        for j in range(horizon.shape[1]):
            print(i, j)
            traces[i, j, :] = amplitude[hrz_idx[j]-top_add:hrz_idx[j]+below_add, i, j]

    return traces

class VAE(nn.Module):
    def __init__(self, hidden_size):
        super(VAE, self).__init__()

        # Encoder
        self.conv1 = nn.Conv1d(2, 3, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(3, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv1d(32, 32, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv1d(32, 32, kernel_size=3, stride=2, padding=1)
        self.fc1 = nn.Linear(256, 128)

        # Latent space
        self.fc21 = nn.Linear(128, hidden_size)
        self.fc22 = nn.Linear(128, hidden_size)

        # Decoder
        self.fc3 = nn.Linear(hidden_size, 128)
        self.fc4 = nn.Linear(128, 256)
        self.deconv1 = nn.ConvTranspose1d(32, 32, kernel_size=4, stride=2, padding=1)
        self.deconv2 = nn.ConvTranspose1d(32, 32, kernel_size=4, stride=2, padding=1)
        self.deconv3 = nn.ConvTranspose1d(32, 32, kernel_size=4, stride=2, padding=1)
        self.conv5 = nn.Conv1d(32, 2, kernel_size=3, stride=1, padding=1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        out = self.relu(self.conv1(x))
        out = self.relu(self.conv2(out))
        out = self.relu(self.conv3(out))
        out = self.relu(self.conv4(out))
        out = out.view(out.size(0), -1)
        h1 = self.relu(self.fc1(out))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            if mu.is_cuda:
                eps = eps.cuda()
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = self.relu(self.fc3(z))

        out = self.relu(self.fc4(h3))

        out = out.view(out.size(0), 32, 8)
        out = self.relu(self.deconv1(out))
        out = self.relu(self.deconv2(out))
        out = self.relu(self.deconv3(out))
        out = self.conv5(out)
        return out

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar, z
    
    
def loss_function(recon_x, x, mu, logvar, window_size=64):
    criterion_mse = nn.MSELoss(size_average=False)
    MSE = criterion_mse(recon_x.view(-1, 2, window_size), x.view(-1, 2, window_size))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return MSE + KLD    
    
# Function to perform one epoch of training
def train(epoch, model, optimizer, train_loader, cuda=False, log_interval=10):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)

        if cuda:
            data = data.cuda()

        optimizer.zero_grad()
        recon_batch, mu, logvar, _ = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item() * data.size(0)
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() * data.size(0) / len(train_loader.dataset)))

    train_loss /= len(train_loader.dataset)
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss))
    return train_loss


# Function to perform evaluation of data on the model, used for testing
def test(epoch, model, test_loader, cuda=False, log_interval=10):
    model.eval()
    test_loss = 0
    with torch.set_grad_enabled(False):
        for i, (data, _) in enumerate(test_loader):
            if cuda:
                data = data.cuda()
            data = Variable(data)
            recon_batch, mu, logvar, _ = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item() * data.size(0)

        test_loss /= len(test_loader.dataset)
        print('====> Test set loss: {:.4f}'.format(test_loss))
    return test_loss


# Function to forward_propagate a set of tensors and receive back latent variables and reconstructions
def forward_all(model, all_loader, cuda=False):
    model.eval()
    reconstructions, latents = [], []
    with torch.set_grad_enabled(False):
        for i, (data, _) in enumerate(all_loader):
            if cuda:
                data = data.cuda()
            data = Variable(data)
            recon_batch, mu, logvar, z = model(data)
            reconstructions.append(recon_batch.cpu())
            latents.append(z.cpu())
    return torch.cat(reconstructions, 0), torch.cat(latents, 0)

### MY Classses

In [18]:
class DataHolder:
    def __init__(self, field_name, inlines, xlines):
        
        # User input attributes
        self.field_name = field_name
        self.inlines = inlines
        self.xlines = xlines

        # near and far offset seismic data
        self.near = None
        self.far = None
        self.twt = None
        
        # Dictionaries for multiple possible entries
        self.horizons = {}
        self.wells = {}
        
        
    def add_segy(self, name, fname):
        if name == 'near':
            self.near, twt = load_seismic(fname, inlines=self.inlines, xlines=self.xlines)
        elif name == 'far':
            self.far, twt = load_seismic(fname, inlines=self.inlines, xlines=self.xlines)
        else:
            raise Exception('please specify if near or far data')
            
        if self.twt is None:
            self.twt = twt
        else: assert (self.twt == twt).all, "This twt does not match the twt from the previous segy"
        
    def add_horizon(self, horizon_name, fname):
        self.horizons[horizon_name] = interpolate_horizon(load_horizon(fname, inlines=self.inlines, xlines=self.xlines))
        
    def add_well(self, well_id, well_i, well_x):
        self.wells[well_id] = [well_i, well_x]
        
        
class Processor:
    def __init__(self, near=None, far=None, twt=None):
        self.raw = [near, far]
        self.twt = twt
        self.out = None
    
    def flatten(self, data, horizon, top_add=12, below_add=52):
        out = []
        for amplitude in data:
            traces = np.zeros((horizon.shape[0], horizon.shape[1], top_add+below_add))
            for i in range(horizon.shape[0]):
                hrz_idx = [np.abs(self.twt-val).argmin() for val in horizon[i, :]] 
                for j in range(horizon.shape[1]):
                    traces[i, j, :] = amplitude[hrz_idx[j]-top_add:hrz_idx[j]+below_add, i, j]
            out.append(traces)
        return out
    
    def normalise(self, data, well_i=38, well_x=138):
        out = []
        for i in data:
            well_variance = np.mean(np.std(i[well_i - 2:well_i + 1, well_x - 2:well_x + 1], 2))
            i /= well_variance
            out.append(i)

        return out
        

    def to_2d(self, data):
        return [i.reshape(-1, data[0].shape[-1]) for i in data]
        
    def average_neighbours(self, neighbours=10):
        return 'not implemented yet'
        
    
    def stack_traces(self, data):
        return np.concatenate([i for i in data], 1)
    
    @property
    def FF(self):
        x_avo = self.out[0]
        y_avo = self.out[1] - self.out[0]

        lin_reg = LinearRegression(fit_intercept=False, normalize=False, copy_X=True, n_jobs=1)
        lin_reg.fit(x_avo.reshape(-1, 1), y_avo.reshape(-1, 1))

        print("Linear Regression coefficient: %1.2f" % lin_reg.coef_[0, 0])
        return y_avo - lin_reg.coef_ * x_avo
    
    def __call__(self, flatten=False, normalise=False, label='FF'):
        self.out = copy.copy(self.raw)
        
        if flatten:
            self.out = self.flatten(self.out, flatten[0], flatten[1], flatten[2])
        if normalise:
            self.out = self.normalise(self.out, normalise[0], normalise[1])
        
        # Flatten arrays from 3d to 2d
        self.out = self.to_2d(self.out)
        
        # Pre-stacking calculation of FF
        if label == 'FF':
            self.label = self.FF
            
        # Stack the traces for output
        self.out = self.stack_traces(self.out)
        
        return self.out, self.label    

In [19]:
class ModelAgent:
    def __init__(self, data):
        self.input = data[0]
        self.label = data[1]
        print("ModelAgent initialised")
        
    def plot_2d(self, data, label):
        fig, ax = plt.subplots(1, 1, figsize=(12, 12))
        sc = ax.scatter(data[:, 0], data[:, 1], s=2.0, c=np.min(label, 1))
#         colorbar(sc)
        return 1
        
    def plot_3d(self, data, feature):
        return 'Not implemented'
        
        
class UMAP(ModelAgent):
    def __init__(self, data):
        super().__init__(data)

    
    def reduce(self, n_neighbors = 50, min_dist=0.001):
        embedding_stack_ff = umap.UMAP(n_neighbors=n_neighbors,
                      min_dist=min_dist,
                      metric='correlation', 
                               verbose=False,
#   this was in ASAP notebook: random_state=42).fit_transform(np.concatenate([stacked, FF.reshape(-1, 64)], 1))
                            random_state=42).fit_transform(self.input)
        
        return embedding_stack_ff
    
    
    
class VAE_model(ModelAgent):
    def __init__(self, data):
        super().__init__(data)
    
    def create_dataloader(self, batch_size=32):
        # split the concatenated input back into two arrays
        X = torch.from_numpy(np.stack(np.split(self.input, 2, axis=1), 1)).float()
        # Create a stacked representation and a zero tensor so we can use the standard Pytorch TensorDataset
        y = torch.from_numpy(np.zeros((X.shape[0], 1))).float()

        print('adf', X.shape)
        
        split = ShuffleSplit(n_splits=1, test_size=0.5)
        for train_index, test_index in split.split(X):
            X_train, y_train = X[train_index], y[train_index]
            X_test, y_test = X[test_index], y[test_index]

        train_dset = TensorDataset(X_train, y_train)
        test_dset = TensorDataset(X_test, y_test)
        all_dset = TensorDataset(X, y)

        kwargs = {'num_workers': 1, 'pin_memory': True}
        self.train_loader = torch.utils.data.DataLoader(train_dset, batch_size=batch_size, shuffle=True, **kwargs)
        self.test_loader = torch.utils.data.DataLoader(test_dset, batch_size=batch_size, shuffle=False, **kwargs)
        self.all_loader = torch.utils.data.DataLoader(all_dset, batch_size=batch_size, shuffle=False, **kwargs)
        
    def train_vae(self, cuda=False, epochs=30):
        set_seed(42)  # Set the random seed
        self.model = VAE(hidden_size=8)  # Inititalize the model

        # use cuda if chosen
        if cuda:
            self.model.cuda()

        # Create a gradient descent optimizer
        optimizer = optim.Adam(self.model.parameters(), lr=1e-2, betas=(0.9, 0.999))

        # Store and plot losses
        self.losses = []

        # Start training loop
        for epoch in range(1, epochs + 1):
            tl = train(epoch, self.model, optimizer, self.train_loader, cuda=False)  # Train model on train dataset
            testl = test(epoch, self.model, self.test_loader, cuda=False)  # Validate model on test dataset
            self.losses.append([tl, testl])
        
    def run_vae(self):
        _, self.zs = forward_all(self.model, self.all_loader, cuda=False)
        
    def vae_umap(self):
        transformer = umap.UMAP(n_neighbors=5,
                                min_dist=0.001,
                                metric='correlation', verbose=True).fit(self.zs.numpy())
        embedding = transformer.transform(self.zs.numpy())
        print('shape of zs', self.zs.shape)

        # plot umap
        fig, ax = plt.subplots(1, 1, figsize=(12, 12))
        sc = ax.scatter(embedding[::, 0], embedding[::, 1], s=2.0, c=np.min(self.label, 1)[::])
        plt.show()
    
    def reduce(self, alpha=0.01):
        self.create_dataloader()
        self.train_vae()
        self.run_vae()
        self.vae_umap()



In [20]:
### Client loader
Glitne = DataHolder("Glitne", [1300, 1502, 2], [1500, 2002, 2])
Glitne.add_segy('near', '../data/3d_nearstack.sgy');
Glitne.add_segy('far', '../data/3d_farstack.sgy');
Glitne.add_horizon('top_heimdal', '../data/Top_Heimdal_subset.txt')
Glitne.add_well('well_1', 36, 276//2)


SegyPY0.57:  readSegy : Trying to read ../data/3d_nearstack.sgy
SegyPY0.57:  getSegyHeader : succesfully read ../data/3d_nearstack.sgy
SegyPY0.57:  filesize=31438840
SegyPY0.57:  bps=    4
SegyPY0.57:  nd=7858810
SegyPY0.57:  readSegyData : Reading segy data
SegyPY0.57:  readSegyData : SEG-Y revision = 0
SegyPY0.57:  readSegyData : DataSampleFormat=1(IBM Float)
SegyPY0.57:   ns=250
SegyPY0.57:  readSegyData : Finished reading segy data
SegyPY0.57:  readSegy : Trying to read ../data/3d_farstack.sgy
SegyPY0.57:  getSegyHeader : succesfully read ../data/3d_farstack.sgy
SegyPY0.57:  filesize=31438840
SegyPY0.57:  bps=    4
SegyPY0.57:  nd=7858810
SegyPY0.57:  readSegyData : Reading segy data
SegyPY0.57:  readSegyData : SEG-Y revision = 0
SegyPY0.57:  readSegyData : DataSampleFormat=1(IBM Float)
SegyPY0.57:   ns=250
SegyPY0.57:  readSegyData : Finished reading segy data


In [21]:
### Client data factory - would run diffetent instances of Processing with different operations
# instance of processing creates options for many outputs
Data = Processor(Glitne.near, Glitne.far, Glitne.twt)
processing_a = Data([Glitne.horizons['top_heimdal'], 12,52], [10, 20])
processing_b = Data([Glitne.horizons['top_heimdal'], 10,10], [10, 20])


### Client model creator/run - run many different instances of the VAE with different parameters
# an instance of a model is one model of dim reduction
UMAP_a = UMAP(processing_a)
# UMAP_a1 = UMAP_a.reduce(n_neighbors=10)
# UMAP_a2 = UMAP_a.reduce(n_neighbors=100)



Linear Regression coefficient: -0.87
Linear Regression coefficient: -0.86
ModelAgent initialised


In [ ]:
# give data once
VAE_a = VAE_model(processing_a)

# run with different params
VAE_a.create_dataloader()
VAE_a.train_vae()
VAE_a.run_vae()
VAE_a.vae_umap()

ModelAgent initialised
adf torch.Size([25351, 2, 64])
Train Epoch: 1 [0/12675 (0%)]	Loss: 10.420815


/anaconda3/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Train Epoch: 1 [320/12675 (3%)]	Loss: 9.607184
Train Epoch: 1 [640/12675 (5%)]	Loss: 9.590000
Train Epoch: 1 [960/12675 (8%)]	Loss: 9.452657
Train Epoch: 1 [1280/12675 (10%)]	Loss: 9.294279
Train Epoch: 1 [1600/12675 (13%)]	Loss: 9.683429
Train Epoch: 1 [1920/12675 (15%)]	Loss: 9.892874
Train Epoch: 1 [2240/12675 (18%)]	Loss: 10.079040
Train Epoch: 1 [2560/12675 (20%)]	Loss: 9.055381
Train Epoch: 1 [2880/12675 (23%)]	Loss: 9.133446
Train Epoch: 1 [3200/12675 (25%)]	Loss: 9.035822
Train Epoch: 1 [3520/12675 (28%)]	Loss: 9.195142
Train Epoch: 1 [3840/12675 (30%)]	Loss: 9.923202
Train Epoch: 1 [4160/12675 (33%)]	Loss: 9.074526
Train Epoch: 1 [4480/12675 (35%)]	Loss: 9.068188
Train Epoch: 1 [4800/12675 (38%)]	Loss: 9.543486
Train Epoch: 1 [5120/12675 (40%)]	Loss: 9.632841
Train Epoch: 1 [5440/12675 (43%)]	Loss: 8.731298
Train Epoch: 1 [5760/12675 (45%)]	Loss: 9.192215
Train Epoch: 1 [6080/12675 (48%)]	Loss: 9.302437
Train Epoch: 1 [6400/12675 (50%)]	Loss: 9.719193
Train Epoch: 1 [6720/1267

Train Epoch: 5 [960/12675 (8%)]	Loss: 9.351937
Train Epoch: 5 [1280/12675 (10%)]	Loss: 9.434128
Train Epoch: 5 [1600/12675 (13%)]	Loss: 9.116040
Train Epoch: 5 [1920/12675 (15%)]	Loss: 10.240353
Train Epoch: 5 [2240/12675 (18%)]	Loss: 9.115788
Train Epoch: 5 [2560/12675 (20%)]	Loss: 9.371310
Train Epoch: 5 [2880/12675 (23%)]	Loss: 9.479669
Train Epoch: 5 [3200/12675 (25%)]	Loss: 9.713393
Train Epoch: 5 [3520/12675 (28%)]	Loss: 9.929147
Train Epoch: 5 [3840/12675 (30%)]	Loss: 8.918063
Train Epoch: 5 [4160/12675 (33%)]	Loss: 9.530819
Train Epoch: 5 [4480/12675 (35%)]	Loss: 9.632721
Train Epoch: 5 [4800/12675 (38%)]	Loss: 9.610268
Train Epoch: 5 [5120/12675 (40%)]	Loss: 9.452918
Train Epoch: 5 [5440/12675 (43%)]	Loss: 9.430332
Train Epoch: 5 [5760/12675 (45%)]	Loss: 8.995875
Train Epoch: 5 [6080/12675 (48%)]	Loss: 9.863892
Train Epoch: 5 [6400/12675 (50%)]	Loss: 9.323079
Train Epoch: 5 [6720/12675 (53%)]	Loss: 9.717957
Train Epoch: 5 [7040/12675 (55%)]	Loss: 9.603884
Train Epoch: 5 [7360/

Train Epoch: 9 [1600/12675 (13%)]	Loss: 9.273346
Train Epoch: 9 [1920/12675 (15%)]	Loss: 9.523267
Train Epoch: 9 [2240/12675 (18%)]	Loss: 9.516146
Train Epoch: 9 [2560/12675 (20%)]	Loss: 10.225354
Train Epoch: 9 [2880/12675 (23%)]	Loss: 9.272872
Train Epoch: 9 [3200/12675 (25%)]	Loss: 9.204021
Train Epoch: 9 [3520/12675 (28%)]	Loss: 10.155710
Train Epoch: 9 [3840/12675 (30%)]	Loss: 9.380068
Train Epoch: 9 [4160/12675 (33%)]	Loss: 8.651577
Train Epoch: 9 [4480/12675 (35%)]	Loss: 8.824230
Train Epoch: 9 [4800/12675 (38%)]	Loss: 9.434975
Train Epoch: 9 [5120/12675 (40%)]	Loss: 10.334941
Train Epoch: 9 [5440/12675 (43%)]	Loss: 9.084042
Train Epoch: 9 [5760/12675 (45%)]	Loss: 9.491182
Train Epoch: 9 [6080/12675 (48%)]	Loss: 8.515174
Train Epoch: 9 [6400/12675 (50%)]	Loss: 9.156331
Train Epoch: 9 [6720/12675 (53%)]	Loss: 9.328277
Train Epoch: 9 [7040/12675 (55%)]	Loss: 9.736135
Train Epoch: 9 [7360/12675 (58%)]	Loss: 9.513989
Train Epoch: 9 [7680/12675 (60%)]	Loss: 9.043984
Train Epoch: 9 [8

Train Epoch: 13 [1280/12675 (10%)]	Loss: 9.598447
Train Epoch: 13 [1600/12675 (13%)]	Loss: 9.657479
Train Epoch: 13 [1920/12675 (15%)]	Loss: 9.609037
Train Epoch: 13 [2240/12675 (18%)]	Loss: 8.627039
Train Epoch: 13 [2560/12675 (20%)]	Loss: 9.664663
Train Epoch: 13 [2880/12675 (23%)]	Loss: 9.321458
Train Epoch: 13 [3200/12675 (25%)]	Loss: 9.162573
Train Epoch: 13 [3520/12675 (28%)]	Loss: 9.073551
Train Epoch: 13 [3840/12675 (30%)]	Loss: 9.469318
Train Epoch: 13 [4160/12675 (33%)]	Loss: 9.751798
Train Epoch: 13 [4480/12675 (35%)]	Loss: 9.393793
Train Epoch: 13 [4800/12675 (38%)]	Loss: 9.755821
Train Epoch: 13 [5120/12675 (40%)]	Loss: 9.412276
Train Epoch: 13 [5440/12675 (43%)]	Loss: 9.313409
Train Epoch: 13 [5760/12675 (45%)]	Loss: 8.879054
Train Epoch: 13 [6080/12675 (48%)]	Loss: 9.286445
Train Epoch: 13 [6400/12675 (50%)]	Loss: 9.339660
Train Epoch: 13 [6720/12675 (53%)]	Loss: 9.551836
Train Epoch: 13 [7040/12675 (55%)]	Loss: 9.948262
Train Epoch: 13 [7360/12675 (58%)]	Loss: 9.696554
